In [38]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from scipy.stats import chi2_contingency

## Tratamiento y exporacion de la data, en este caso se construiran 3 diferentes modelos para cual trabajaremos de 3 diferentes formas con la data
* 1 Se hara uso de one hot encoding en todas las columnas
* 2 Se le asignara un valor numerico para cada variable categorica
* 3 Se normalizara la data del punto 2

# Exploracion de la data

In [39]:
data = pd.read_csv("training-ds.csv")
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 964 entries, 0 to 963
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Tipo de Red               964 non-null    object
 1   Estudiante de Tecnología  964 non-null    object
 2   Nivel de Educación        964 non-null    object
 3   Vive en Ciudad            964 non-null    object
 4   Tipo de Instituto         964 non-null    object
 5   Edad                      964 non-null    object
 6   Dispositivo               964 non-null    object
 7   Tipo de Internet          964 non-null    object
 8   Situación Financiera      964 non-null    object
 9   Género                    964 non-null    object
 10  Duración de la Clase      964 non-null    object
 11  Nivel de Adaptación       964 non-null    object
dtypes: object(12)
memory usage: 90.5+ KB
None


Como se puede observar, al pedir la informacion de la data, nos podemos percatar que todas las columnas, son del tipo 'Object', lo cual indica que son del tipo de datos de texto

In [40]:
data.shape

(964, 12)

Aca tambien vemos algo basico como lo es la cantidad de filas y de columnas que tiene el dataset, en este caso de 964 filas y 12 columnas

In [41]:
def print_unique_values(data_frame, dtype='object'):
    for column in data_frame.columns:
        if data_frame[column].dtype == dtype:
            print(f'{column}: {list(data_frame[column].unique())}')


print_unique_values(data)

Tipo de Red: ['3G', '4G', '2G']
Estudiante de Tecnología: ['Si', 'No']
Nivel de Educación: ['Universidad', 'Escuela', 'Colegio']
Vive en Ciudad: ['Si', 'No']
Tipo de Instituto: ['Privado', 'Público']
Edad: ['21-25', 'Nov-15', '16-20', '26-30', '01-May', '06-Oct']
Dispositivo: ['Computadora', 'Smartphone', 'Tableta']
Tipo de Internet: ['Wifi', 'Compra Megas']
Situación Financiera: ['Media', 'Mala', 'Buena']
Género: ['Masculino', 'Femenino']
Duración de la Clase: ['03-Jun', '01-Mar', '0']
Nivel de Adaptación: ['Bajo', 'Moderado', 'Alto']


Aca lo que se hace es tener un avision general de los valores únicos en cada columna del conjunto de datos.

Aca lo que se pudo observar es que hay algunas fechas de edad y de duracion de la clase que tienen letras como Nov, etc.


## Columnas relevantes para los distintos casos

Buscando que columnas son relevantes  para nuestro modelo en el caso 1

In [ ]:
data_transform= data
data_transform = pd.get_dummies(data, columns=[ 'Estudiante de Tecnología', "Vive en Ciudad",'Tipo de Instituto','Tipo de Internet','Género'])

Lo primero que hacemos aca es una copia del data set para luego aplicarle get_dummies, esta funcion se caracteriza por realizar one-hot de las variables categoricas. Esto significa que en este caso creara columnas adicionales para 'Estudiante de Texnologia', 'Vive en Ciudad', 'Tipo de Instituto', y  'Genero'.

In [49]:
correlation_level_adaption = data_transform.corrwith(data_transform['Nivel de Adaptación'], method='kendall')
correlation_level_adaption = correlation_level_adaption.abs().sort_values(ascending=False)
print(correlation_level_adaption.head(11))


Nivel de Adaptación              1.000000
Duración de la Clase             0.240134
Tipo de Instituto_Privado        0.205034
Tipo de Instituto_Público        0.205034
Situación Financiera             0.201734
Edad                             0.167064
Vive en Ciudad_No                0.136796
Vive en Ciudad_Si                0.136796
Estudiante de Tecnología_Si      0.064079
Estudiante de Tecnología_No      0.064079
Tipo de Internet_Compra Megas    0.049212
dtype: float64


Con la funcion corrwith calculamos la correlacion de cada columna con la columna 'nivel de adaptacion', esto se logra porque aplicamos el metodo de correlacion de Kendall, lo que hace este medoto es evaluar la asociación entre dos variables clasificadas.

La funcion abs() es para tratar con la magnitud de la correlación  y tambien los ordenamos en orden descendente.

Para finalizar podemos ver como es la correlacion de cada columna con la columna de nivel de adaptacion, con esto nos damos cuenta que los unicos que parecen tener el nivel mas alto son "Duracion de la clase", "Tipo de Instituto_Privado", "Tipo de Instituto_Público", "Situación Financiera ", sin embargo apra este primer caso, utilizaremos todas las columnas aplicandoles one hot encoding.

Buscando que columnas son relevantes  para nuestro modelo en el caso 2

In [50]:
correlation_level_adaption = data_transform.corrwith(data_transform['Nivel de Adaptación'], method='kendall')
correlation_level_adaption = correlation_level_adaption.abs().sort_values(ascending=False)
print(correlation_level_adaption.head(11))


Nivel de Adaptación              1.000000
Duración de la Clase             0.240134
Tipo de Instituto_Privado        0.205034
Tipo de Instituto_Público        0.205034
Situación Financiera             0.201734
Edad                             0.167064
Vive en Ciudad_No                0.136796
Vive en Ciudad_Si                0.136796
Estudiante de Tecnología_Si      0.064079
Estudiante de Tecnología_No      0.064079
Tipo de Internet_Compra Megas    0.049212
dtype: float64


# Procesamiento de la data

In [42]:
def convertir_mes_a_numero(fecha):
    if len(fecha) == 5 and fecha[1] == '-' and fecha[2:].isdigit():
        return fecha  
    
    meses = {
        'Jan': '1', 'Feb': '2', 'Mar': '3', 'Apr': '4',
        'May': '5', 'Jun': '6', 'Jul': '7', 'Aug': '8',
        'Sep': '9', 'Oct': '10', 'Nov': '11', 'Dec': '12'
    }
    if len(fecha) == 6 and fecha[2] == '-' and fecha[:2].isdigit():
        numero_mes = meses.get(fecha[3:])
        if numero_mes:
            return f"{int(fecha[:2])}-{numero_mes.zfill(2)}"
    
    partes = fecha.split('-')
    numero_mes = meses.get(partes[0])
    if numero_mes:
        return f"{numero_mes}-{partes[1]}"
    else:
        return fecha 

data['Edad'] = data['Edad'].apply(convertir_mes_a_numero)
data['Duración de la Clase'] = data['Duración de la Clase'].apply(convertir_mes_a_numero)

En esta funcion nos concentramos en cambiar y arreglar los datos de las columnas "Edad" y "Duracion de la clase", convertimos los meses en datos numericos para un mejor manejo de la data y que no haya errores.

A continuacion se muestra es resultado.

In [63]:
data

,Tipo de Red,Estudiante de Tecnología,Nivel de Educación,Vive en Ciudad,Tipo de Instituto,Edad,Dispositivo,Tipo de Internet,Situación Financiera,Género,Duración de la Clase,Nivel de Adaptación
0,3G,Si,Universidad,Si,Privado,21-25,Computadora,Wifi,Media,Masculino,3-06,0
1,3G,No,Escuela,Si,Privado,11-15,Smartphone,Compra Megas,Media,Femenino,1-03,1
2,3G,Si,Universidad,Si,Privado,21-25,Smartphone,Compra Megas,Mala,Masculino,1-03,0
3,3G,Si,Escuela,Si,Privado,11-15,Smartphone,Compra Megas,Media,Masculino,1-03,1
4,4G,Si,Universidad,Si,Privado,21-25,Computadora,Wifi,Buena,Masculino,1-03,2
...,...,...,...,...,...,...,...,...,...,...,...,...
959,3G,No,Universidad,Si,Privado,21-25,Smartphone,Compra Megas,Media,Masculino,1-03,1
960,3G,No,Escuela,Si,Privado,11-15,Smartphone,Compra Megas,Media,Femenino,1-03,1
961,4G,No,Universidad,Si,Público,21-25,Smartphone,Compra Megas,Media,Masculino,0,0
962,3G,No,Universidad,Si,Privado,21-25,Smartphone,Compra Megas,Media,Masculino,1-03,1


Manejo de la data tipo 1

In [64]:
#ESTO NO SE SI HAY QUE BORRARLO
data_transform= data
data_transform = pd.get_dummies(data, columns=[ 'Estudiante de Tecnología', "Vive en Ciudad",'Tipo de Instituto','Tipo de Internet','Género'])
display(data_transform.head(100))

,Tipo de Red,Nivel de Educación,Edad,Dispositivo,Situación Financiera,Duración de la Clase,Nivel de Adaptación,Estudiante de Tecnología_No,Estudiante de Tecnología_Si,Vive en Ciudad_No,Vive en Ciudad_Si,Tipo de Instituto_Privado,Tipo de Instituto_Público,Tipo de Internet_Compra Megas,Tipo de Internet_Wifi,Género_Femenino,Género_Masculino
0,3G,Universidad,21-25,Computadora,Media,3-06,0,False,True,False,True,True,False,False,True,False,True
1,3G,Escuela,11-15,Smartphone,Media,1-03,1,True,False,False,True,True,False,True,False,True,False
2,3G,Universidad,21-25,Smartphone,Mala,1-03,0,False,True,False,True,True,False,True,False,False,True
3,3G,Escuela,11-15,Smartphone,Media,1-03,1,False,True,False,True,True,False,True,False,False,True
4,4G,Universidad,21-25,Computadora,Buena,1-03,2,False,True,False,True,True,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4G,Colegio,16-20,Smartphone,Media,0,0,False,True,True,False,False,True,True,False,False,True
96,4G,Escuela,16-20,Computadora,Media,1-03,0,True,False,False,True,True,False,False,True,True,False
97,4G,Universidad,21-25,Smartphone,Media,0,0,True,False,False,True,True,False,True,False,False,True
98,3G,Universidad,21-25,Smartphone,Media,1-03,0,True,False,False,True,False,True,True,False,False,True


In [65]:
mapping = {'Bajo': 0, 'Moderado': 1, 'Alto': 2}
data['Nivel de Adaptación'] = data['Nivel de Adaptación'].map(mapping)

print(data['Nivel de Adaptación'])


0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
959   NaN
960   NaN
961   NaN
962   NaN
963   NaN
Name: Nivel de Adaptación, Length: 964, dtype: float64


Para nuestro primer caso, lo que hacemos es, los datos de la columna de adaptacion, les damos valores numericos, en este caso que van desde 0 a 2, relacionados desde nivel de apatacion bajo a nivel de adaptacion alto respectivamente.

### Manejo de la data tipo 2

In [47]:
data_encoded = data.apply(LabelEncoder().fit_transform)
display(data_encoded.head(100))

,Tipo de Red,Estudiante de Tecnología,Nivel de Educación,Vive en Ciudad,Tipo de Instituto,Edad,Dispositivo,Tipo de Internet,Situación Financiera,Género,Duración de la Clase,Nivel de Adaptación
0,1,1,2,1,0,3,0,1,2,1,2,1
1,1,0,1,1,0,1,1,0,2,0,1,2
2,1,1,2,1,0,3,1,0,1,1,1,1
3,1,1,1,1,0,1,1,0,2,1,1,2
4,2,1,2,1,0,3,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,2,1,0,0,1,2,1,0,2,1,0,1
96,2,0,1,1,0,2,0,1,2,0,1,1
97,2,0,2,1,0,3,1,0,2,1,0,1
98,1,0,2,1,1,3,1,0,2,1,1,1


### Data tipo 3

In [48]:
# scaler = StandardScaler()
# df_normalized_array = scaler.fit_transform(df_encoded)
# data_normalice = pd.DataFrame(df_normalized_array, columns=df_encoded.columns)


# display(data_normalice.head(100))

In [62]:
correlation_nivel_adaptacion_encod = data_encoded.corr(method='pearson')['Nivel de Adaptación']
display((correlation_nivel_adaptacion_encod[np.argsort(correlation_nivel_adaptacion_encod, axis=0)[::-1]]).head(11))

Nivel de Adaptación         1.000000
Situación Financiera        0.258934
Duración de la Clase        0.209937
Vive en Ciudad              0.104525
Dispositivo                 0.066690
Estudiante de Tecnología    0.044715
Tipo de Red                 0.021874
Nivel de Educación         -0.015269
Género                     -0.042976
Tipo de Internet           -0.069489
Tipo de Instituto          -0.172572
Name: Nivel de Adaptación, dtype: float64

Buscando que columnas son relevantes  para nuestro modelo en el caso 3

In [52]:
# correlation_nivel_adaptacion_norm = data_normalice.corr(method='pearson')['Nivel de Adaptación']
# display((correlation_nivel_adaptacion_norm[np.argsort(correlation_nivel_adaptacion_norm, axis=0)[::-1]]).head(11))

# Training

In [57]:
x = data.drop('Nivel de Adaptación' , axis = 1)
y = data['Nivel de Adaptación']
x

,Tipo de Red,Estudiante de Tecnología,Nivel de Educación,Vive en Ciudad,Tipo de Instituto,Edad,Dispositivo,Tipo de Internet,Situación Financiera,Género,Duración de la Clase
0,3G,Si,Universidad,Si,Privado,21-25,Computadora,Wifi,Media,Masculino,3-06
1,3G,No,Escuela,Si,Privado,11-15,Smartphone,Compra Megas,Media,Femenino,1-03
2,3G,Si,Universidad,Si,Privado,21-25,Smartphone,Compra Megas,Mala,Masculino,1-03
3,3G,Si,Escuela,Si,Privado,11-15,Smartphone,Compra Megas,Media,Masculino,1-03
4,4G,Si,Universidad,Si,Privado,21-25,Computadora,Wifi,Buena,Masculino,1-03
...,...,...,...,...,...,...,...,...,...,...,...
959,3G,No,Universidad,Si,Privado,21-25,Smartphone,Compra Megas,Media,Masculino,1-03
960,3G,No,Escuela,Si,Privado,11-15,Smartphone,Compra Megas,Media,Femenino,1-03
961,4G,No,Universidad,Si,Público,21-25,Smartphone,Compra Megas,Media,Masculino,0
962,3G,No,Universidad,Si,Privado,21-25,Smartphone,Compra Megas,Media,Masculino,1-03


En esta seccion, lo que hacemos es primero que nada dividir el data set en dos secciones, 'x' y 'y' en x guardaremos todas las columnas del data set, excepto la columna del nivel de adaptacion que es la que queremos comparar.

En 'y' se guardara solo la columna de nivel de adaptacion. 

In [70]:

x_train , x_test , y_train , y_test = train_test_split(pd.get_dummies(x) , y , test_size = 0.2 , random_state = 42)


Luego dividimos el data set de la siguiente manera: 
    
    - Para entrenar el data set "x_train"= lo que hacemos es on hot encoding de todas las columnas que fueron seleciconadas antes.
    - "x_test" = y, lo cual sera nuestra columna objetivo con los valores de nivel de adaptacion
    - "y_train" = "test_size = 0.2", significa que solo usaremos el 20% de los datos para prueba
    - "y_test" = "random_state = 42" Nos da una semilla para garantizar que la division sea la misma cada vez que ejecutamos el codigo

In [77]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape = x_train.shape[1:]),
    tf.keras.layers.Dense(100 , activation = 'relu'),
    tf.keras.layers.Dense(100 , activation = 'relu'),
    tf.keras.layers.Dense(100 , activation = 'relu'),
    tf.keras.layers.Dense(100 , activation = 'relu'),
    tf.keras.layers.Dense(3 , activation = 'softmax')
])

model.compile(loss = tf.keras.losses.sparse_categorical_crossentropy , optimizer = tf.keras.optimizers.Adam() , metrics = ['accuracy'])

model.fit(x_train , y_train , epochs = 200)

Epoch 1/200
25/25 [==============================] - 1s 1ms/step - loss: 0.8887 - accuracy: 0.5772
Epoch 2/200
25/25 [==============================] - 0s 1ms/step - loss: 0.7364 - accuracy: 0.6732
Epoch 3/200
25/25 [==============================] - 0s 1ms/step - loss: 0.6438 - accuracy: 0.7328
Epoch 4/200
25/25 [==============================] - 0s 988us/step - loss: 0.5678 - accuracy: 0.7678
Epoch 5/200
25/25 [==============================] - 0s 1ms/step - loss: 0.5361 - accuracy: 0.7821
Epoch 6/200
25/25 [==============================] - 0s 1ms/step - loss: 0.4820 - accuracy: 0.8054
Epoch 7/200
25/25 [==============================] - 0s 1ms/step - loss: 0.4281 - accuracy: 0.8249
Epoch 8/200
25/25 [==============================] - 0s 1ms/step - loss: 0.4264 - accuracy: 0.8197
Epoch 9/200
25/25 [==============================] - 0s 1ms/step - loss: 0.4096 - accuracy: 0.8236
Epoch 10/200
25/25 [==============================] - 0s 2ms/step - loss: 0.3644 - accuracy: 0.8586
Epoch 1

In [78]:
loss , acc = model.evaluate(x_test , y_test)
acc

7/7 [==============================] - 0s 1ms/step - loss: 0.3479 - accuracy: 0.8497


0.8497409224510193

### Construccion del modelo para el caso 2

In [ ]:

X_encod = data_encoded.drop('Nivel de Adaptación' , axis = 1)
y_encod = data_encoded['Nivel de Adaptación']
x_train_encod , x_test_encod , y_train_encod , y_test_encod = train_test_split(X_encod , y_encod , test_size = 0.2 , random_state = 42)
model_encode = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape = x_train_encod.shape[1:]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(33 , activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(66 , activation = "relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2) ,
    tf.keras.layers.Dense(3 , activation = 'softmax')
])

# Compilar el modelo
model_encode.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

# Entrenar el modelo con callbacks
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model_encode.fit(x_train_encod, y_train_encod, epochs=100, validation_split=0.2, batch_size=64, callbacks=[callback])


test_loss, test_accuracy = model_encode.evaluate(x_test_encod, y_test_encod)
print(f'Test accuracy: {test_accuracy}')



Epoch 1/100
10/10 [==============================] - 2s 40ms/step - loss: 1.2434 - accuracy: 0.4107 - val_loss: 1.0464 - val_accuracy: 0.4194
Epoch 2/100
10/10 [==============================] - 0s 8ms/step - loss: 1.0333 - accuracy: 0.5292 - val_loss: 1.0137 - val_accuracy: 0.4323
Epoch 3/100
10/10 [==============================] - 0s 8ms/step - loss: 0.9514 - accuracy: 0.5974 - val_loss: 0.9891 - val_accuracy: 0.4452
Epoch 4/100
10/10 [==============================] - 0s 8ms/step - loss: 0.8361 - accuracy: 0.6542 - val_loss: 0.9675 - val_accuracy: 0.4581
Epoch 5/100
10/10 [==============================] - 0s 7ms/step - loss: 0.8215 - accuracy: 0.6656 - val_loss: 0.9505 - val_accuracy: 0.5290
Epoch 6/100
10/10 [==============================] - 0s 9ms/step - loss: 0.7468 - accuracy: 0.6883 - val_loss: 0.9300 - val_accuracy: 0.5419
Epoch 7/100
10/10 [==============================] - 0s 8ms/step - loss: 0.7120 - accuracy: 0.7175 - val_loss: 0.9122 - val_accuracy: 0.5613
Epoch 8/100


In [ ]:
loss_encod , acc_encod = model_encode.evaluate(x_test_encod , y_test_encod)
acc_encod

7/7 [==============================] - 0s 3ms/step - loss: 0.3870 - accuracy: 0.8238


0.8238341808319092

In [ ]:
#guardando el mejor modelo obtenido hasta ahora
#model_dir = './BestModel'
#tf.saved_model.save(model_encode, model_dir)
#model_encode.save('bestAcury86_11_25.h5')

In [ ]:
loaded_model = tf.keras.models.load_model('bestAcury86_11_25.h5')

# Ahora puedes utilizar métodos como evaluate y predict en el modelo cargado
loss_encod, acc_encod = loaded_model.evaluate(x_test_encod, y_test_encod)

7/7 [==============================] - 0s 2ms/step - loss: 0.3944 - accuracy: 0.8601


## CArgando la data con la que realizaremos las prediciones

In [ ]:
data_test = pd.read_csv("submission-ds.csv")
data_test1 =pd.get_dummies(data_test)

In [ ]:
data_test['Edad'] = data_test['Edad'].apply(convertir_mes_a_numero)
data_test['Duración de la Clase'] = data_test['Duración de la Clase'].apply(convertir_mes_a_numero)
data_test = data_test.apply(LabelEncoder().fit_transform)
data_test

TypeError: object of type 'int' has no len()

### Preciendo caso 1

In [ ]:
pedict_m1= model.predict(data_test1)
predict_model1 = pedict_m1.argmax(axis=1)
print(predict_model1)

8/8 [==============================] - 0s 2ms/step
[0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 1 0 1 1 1 0 1 1 1 1 0 0 0 0 1 1 1 0 1 0 0 0
 1 1 0 1 0 0 0 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 0 1 0 0 1 0 1 0 0 0 1 0 1 0 1
 0 1 1 0 1 1 1 1 0 1 1 2 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 2 0 1 1 0 0 1 1 1 1
 0 1 1 0 1 1 1 1 0 1 0 2 0 1 0 1 0 2 0 1 0 1 1 1 1 0 0 0 1 1 1 1 0 1 1 1 1
 2 1 1 0 1 1 1 1 1 1 1 1 0 2 1 1 0 1 2 1 0 1 1 1 1 0 0 1 0 0 1 0 1 0 1 0 0
 1 0 1 1 1 1 1 0 0 0 0 0 0 0 1 0 0 1 1 1 1 1 1 1 1 0 0 1 1 0 1 1 0 1 1 1 0
 0 0 1 1 1 0 1 1 1 0 2 1 0 0 1 1 1 1 1]


In [ ]:
resultados_prediccion1 = pd.DataFrame({
    'id': range(1, len(predict_model1) + 1),
    'Nivel de Adaptación': predict_model1
})
resultados_prediccion1['Nivel de Adaptación'] = resultados_prediccion1['Nivel de Adaptación'].map({0: 'Bajo', 1: 'Moderado', 2: 'Alto'})

print(resultados_prediccion1)
resultados_prediccion1.to_csv('resultados_prediccion_m1.csv', index=False)

      id Nivel de Adaptación
0      1                Bajo
1      2                Bajo
2      3                Bajo
3      4                Bajo
4      5                Bajo
..   ...                 ...
236  237            Moderado
237  238            Moderado
238  239            Moderado
239  240            Moderado
240  241            Moderado

[241 rows x 2 columns]


### Predicion Caso 2

In [ ]:
predictions = loaded_model.predict(data_test)
predicted_classes = predictions.argmax(axis=1)
print(predicted_classes)

8/8 [==============================] - 0s 2ms/step
[2 1 1 1 1 0 1 2 1 1 1 2 2 1 1 2 2 2 2 2 1 1 2 2 2 1 1 1 1 2 2 1 0 2 1 1 1
 2 2 1 2 1 1 1 2 2 2 2 1 2 2 1 2 1 2 2 1 2 2 1 2 1 1 2 1 2 1 1 1 2 1 2 1 2
 1 2 2 1 2 2 2 2 1 2 2 0 1 1 2 2 2 2 2 2 2 2 2 2 2 1 2 0 1 2 2 1 1 2 2 2 2
 1 2 1 1 2 2 2 0 1 2 1 2 1 2 1 2 1 0 1 2 1 2 2 2 2 1 1 1 2 2 2 2 1 2 2 2 1
 0 2 2 1 2 2 2 2 2 2 2 2 1 0 0 2 1 2 0 2 1 2 2 2 2 1 1 2 1 1 2 1 2 1 2 1 1
 2 1 2 2 2 2 2 1 2 2 1 1 1 1 2 1 1 2 2 2 2 2 1 2 2 1 1 2 2 1 2 2 1 1 1 2 1
 1 1 2 2 2 1 2 2 2 1 0 2 1 1 2 2 2 2 2]


In [ ]:
resultados_prediccion = pd.DataFrame({
    'id': range(1, len(predicted_classes) + 1),
    'Nivel de Adaptación': predicted_classes
})
resultados_prediccion['Nivel de Adaptación'] = resultados_prediccion['Nivel de Adaptación'].map({0: 'Alto', 1: 'Bajo', 2: 'Moderado'})

# Mostrar el DataFrame con las etiquetas mapeadas
print(resultados_prediccion)


      id Nivel de Adaptación
0      1            Moderado
1      2                Bajo
2      3                Bajo
3      4                Bajo
4      5                Bajo
..   ...                 ...
236  237            Moderado
237  238            Moderado
238  239            Moderado
239  240            Moderado
240  241            Moderado

[241 rows x 2 columns]


In [ ]:
resultados_prediccion.to_csv('resultados_prediccion_ac86.csv', index=False)